# 1. Package Installation and Import

In [1]:
pip install altair

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install vega_datasets

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install openpyxl

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install vl-convert-python

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import altair as alt
from vega_datasets import data
import pymc as pm
import arviz as az
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.ensemble import RandomForestRegressor
from scipy.interpolate import CubicSpline

fips_to_abbr = {
    1: 'AL', 2: 'AK', 4: 'AZ', 5: 'AR', 6: 'CA', 8: 'CO', 9: 'CT', 10: 'DE', 11: 'DC',
    12: 'FL', 13: 'GA', 15: 'HI', 16: 'ID', 17: 'IL', 18: 'IN', 19: 'IA', 20: 'KS',
    21: 'KY', 22: 'LA', 23: 'ME', 24: 'MD', 25: 'MA', 26: 'MI', 27: 'MN', 28: 'MS',
    29: 'MO', 30: 'MT', 31: 'NE', 32: 'NV', 33: 'NH', 34: 'NJ', 35: 'NM', 36: 'NY',
    37: 'NC', 38: 'ND', 39: 'OH', 40: 'OK', 41: 'OR', 42: 'PA', 44: 'RI', 45: 'SC',
    46: 'SD', 47: 'TN', 48: 'TX', 49: 'UT', 50: 'VT', 51: 'VA', 53: 'WA', 54: 'WV',
    55: 'WI', 56: 'WY'
}

abbr_to_fips = {
    'AL': 1, 'AK': 2, 'AZ': 4, 'AR': 5, 'CA': 6, 'CO': 8,
    'CT': 9, 'DE': 10, 'DC': 11, 'FL': 12, 'GA': 13, 'HI': 15,
    'ID': 16, 'IL': 17, 'IN': 18, 'IA': 19, 'KS': 20, 'KY': 21,
    'LA': 22, 'ME': 23, 'MD': 24, 'MA': 25, 'MI': 26, 'MN': 27,
    'MS': 28, 'MO': 29, 'MT': 30, 'NE': 31, 'NV': 32, 'NH': 33,
    'NJ': 34, 'NM': 35, 'NY': 36, 'NC': 37, 'ND': 38, 'OH': 39,
    'OK': 40, 'OR': 41, 'PA': 42, 'RI': 44, 'SC': 45, 'SD': 46,
    'TN': 47, 'TX': 48, 'UT': 49, 'VT': 50, 'VA': 51, 'WA': 53,
    'WV': 54, 'WI': 55, 'WY': 56
}

state_to_abbr = {
    'Alabama': 'AL', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'District of Columbia': 'DC',
    'Florida': 'FL', 'Georgia': 'GA', 'Idaho': 'ID', 'Illinois': 'IL',
    'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN',
    'Mississippi': 'MS', 'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR',
    'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD',
    'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}

# 2. Datasets Description

$$
\begin{array}{l l}
\hline
\text{Data} & \text{File} \\
\hline
\text{COPD rate}      & \texttt{U.S.\_Chronic\_Disease\_Indicators\_CDI\_2023\_Release\_filtered\_by\_COPD.csv} \\
\text{Smoking rate}   & \texttt{BRFSS\_Prevalence\_Data\_(2011\_to\_present)\_filtered\_by\_smoking.csv} \\
\text{Diabetes rate}  & \texttt{BRFSS\_Prevalence\_Data\_(2011\_to\_present)\_filtered\_by\_diabetes.csv} \\
\text{Poverty rate}   & \texttt{poverty-rate-by-raceethnicity(2008\_to\_2023).csv} \\
\text{PM2.5 level}     & \texttt{Daily\_Census\_Tract-Level\_PM2.5\_2016-2020\_averaged\_by\_state.csv} \\
\hline
\end{array}
$$


### Filtering
```python
# COPD rate: https://www.kaggle.com/datasets/kadiryildirim12/u-s-chronic-disease-indicators-cdi-2023-release/data
U.S._Chronic_Disease_Indicators__CDI___2023_Release_filtered_by_COPD.csv =
    U.S._Chronic_Disease_Indicators__CDI___2023_Release.csv[
    ('Topic'=='Chronic Obstructive Pulmonary Disease') & 
    ('DataSource'=='BRFSS') & 
    ('Question'=='Prevalence of chronic obstructive pulmonary disease among adults >= 18') & 
    ('StratificationCategory1'=='Overall') & 
    ('DataValueType'=='Crude Prevalence')]

# Smoking rate: https://data.cdc.gov/Behavioral-Risk-Factors/Behavioral-Risk-Factor-Surveillance-System-BRFSS-P/dttw-5yxu/about_data
Behavioral_Risk_Factor_Surveillance_System_(BRFSS)_Prevalence_Data_(2011_to_present)
_filtered_by_smoking.csv = 
    Behavioral_Risk_Factor_Surveillance_System_(BRFSS)_Prevalence_Data_(2011_to_present).csv[
    ('Topic'=='Current Smoker Status') & 
    ('Break_Out_Category'=='Overall') & 
    ('Response'=='Yes') & 
    ('Data_value_type'=='Crude Prevalence')]

# Diabetes rate: https://data.cdc.gov/Behavioral-Risk-Factors/Behavioral-Risk-Factor-Surveillance-System-BRFSS-P/dttw-5yxu/about_data
Behavioral_Risk_Factor_Surveillance_System_(BRFSS)_Prevalence_Data_(2011_to_present)
_filtered_by_diabetes.csv = 
    Behavioral_Risk_Factor_Surveillance_System_(BRFSS)_Prevalence_Data_(2011_to_present).csv[
    ('Topic'=='Diabetes') & 
    ('Question'=='Have you ever been told by a doctor that you have diabetes?') & 
    ('Response'=='Yes') & 
    ('Break_Out_Category'=='Overall') & 
    ('Data_value_type'=='Crude Prevalence')]

# Poverty rate: https://www.kff.org/state-health-policy-data/state-indicator/poverty-rate-by-raceethnicity/
poverty-rate-by-raceethnicity(2008_to_2023).csv

# PM2.5 level: https://data.cdc.gov/Environmental-Health-Toxicology/Daily-Census-Tract-Level-PM2-5-Concentrations-2016/96sd-hxdt/about_data
aggregated_chunks = []
for chunk in pd.read_csv('Daily_Census_Tract-Level_PM2.5_Concentrations,_2016_-_2020.csv', 
                         chunksize=100000, thousands=','):
    aggregated_chunks.append(chunk.groupby(['year', 'statefips'])['DS_PM_pred']
        .mean().reset_index())
df_final = pd.concat(aggregated_chunks)
    .groupby(['year', 'statefips'], as_index=False)['DS_PM_pred'].mean()
df_final.to_csv('Daily_Census_Tract-Level_PM2.5_Concentrations,_2016_-_2020\
    _averaged_by_state.csv', index=False)
```

# 3. Visualizations

In [6]:
df = pd.read_csv('U.S._Chronic_Disease_Indicators__CDI___2023_Release_filtered_by_COPD.csv')
charts = []
for yr in range(2011, 2021):
    d = df[df['YearStart'] == yr]
    chart = (
        alt.Chart(alt.topo_feature(data.us_10m.url, 'states'))
        .mark_geoshape()
        .encode(
            color=alt.Color(
                'DataValue:Q',
                scale=alt.Scale(scheme='reds'),
                title='COPD (%)'
            ),
            tooltip=['LocationDesc:N', alt.Tooltip('DataValue:Q', format='.1f')]
        )
        .transform_lookup(lookup='id', from_=alt.LookupData(d, 'LocationID', ['DataValue', 'LocationDesc']))
        .properties(width=180, height=120, title=str(yr))
        .project('albersUsa')
    )
    charts.append(chart)
row_size = 5
grid = alt.vconcat(*[
    alt.hconcat(*charts[i:i+row_size])
    for i in range(0, len(charts), row_size)
])
grid.properties(title='COPD Prevalence by State')

alt.VConcatChart(...)

In [7]:
# COPD data
copd_df = pd.read_csv('U.S._Chronic_Disease_Indicators__CDI___2023_Release_filtered_by_COPD.csv').rename(columns={'DataValue': 'COPD_Rate', 'YearStart': 'Year'})
# Smoking data
smoking_df = pd.read_csv('Behavioral_Risk_Factor_Surveillance_System_(BRFSS)_Prevalence_Data_(2011_to_present)_filtered_by_smoking.csv').rename(columns={'Locationabbr': 'LocationAbbr', 'Data_value': 'Smoking_Rate'})
# Poverty data
poverty_df = pd.read_csv('poverty-rate-by-raceethnicity(2008_to_2023).csv', header=2)
poverty_df = poverty_df.melt(
    id_vars=['Location'],
    value_vars=[c for c in poverty_df.columns if '__Total' in c],
    var_name='Year_Type',
    value_name='Poverty_Rate'
)
poverty_df['Year'] = poverty_df['Year_Type'].str.split('__').str[0].astype(int)
poverty_df['Poverty_Rate'] = poverty_df['Poverty_Rate']*100
poverty_df['LocationAbbr'] = poverty_df['Location'].map(state_to_abbr)
# Diabetes data
diabetes_df = pd.read_csv('Behavioral_Risk_Factor_Surveillance_System_(BRFSS)_Prevalence_Data_(2011_to_present)_filtered_by_diabetes.csv').rename(columns={'Locationabbr': 'LocationAbbr', 'Data_value': 'Diabetes_Rate'})
# PM2.5 data
pm25_df = pd.read_csv('Daily_Census_Tract-Level_PM2.5_Concentrations,_2016_-_2020_averaged_by_state.csv').rename(columns={'year': 'Year', 'DS_PM_pred': 'PM25_Level'})
pm25_df['LocationAbbr'] = pm25_df['statefips'].map(fips_to_abbr)
# Display
merged_df = (copd_df.merge(smoking_df, on=['LocationAbbr', 'Year'], how='inner')
                    .merge(diabetes_df, on=['LocationAbbr', 'Year'], how='inner')
                    .merge(poverty_df, on=['LocationAbbr', 'Year'], how='inner')
                    .merge(pm25_df, on=['LocationAbbr', 'Year'], how='inner'))
display_df = merged_df.rename(columns={
    'COPD_Rate': 'COPD Prevalence (%)',
    'Smoking_Rate': 'Smoking Rate (%)',
    'Diabetes_Rate': 'Diabetes Prevalence (%)', 
    'Poverty_Rate': 'Poverty Rate (%)',
    'PM25_Level': 'PM2,5 Level (μg/m³)'
})
print('low PM2.5 region', display_df[display_df['PM2,5 Level (μg/m³)']<7]['LocationDesc'].unique())
scatter_matrix = alt.Chart(display_df).mark_circle().encode(
    alt.X(alt.repeat("column"), type='quantitative', scale=alt.Scale(zero=False)),
    alt.Y(alt.repeat("row"), type='quantitative', scale=alt.Scale(zero=False)),
    color=alt.Color('Year:O', scale=alt.Scale(scheme='viridis'), 
                   legend=alt.Legend(title='Year')),
    tooltip=['LocationAbbr', 'Year']
).repeat(
    row=['COPD Prevalence (%)'],
    column=['Smoking Rate (%)', 'Diabetes Prevalence (%)', 'Poverty Rate (%)', 'PM2,5 Level (μg/m³)']
).properties(
    title='Correlation between Health Factors, Poverty and PM2.5 Level with COPD Prevalence'
)
corr_cols = [
    'COPD Prevalence (%)',
    'Smoking Rate (%)',
    'Diabetes Prevalence (%)',
    'Poverty Rate (%)',
    'PM2,5 Level (μg/m³)'
]
correlation_matrix = display_df[corr_cols].corr(method='pearson')
print("=== Correlation Matrix (Pearson) ===")
display(correlation_matrix.round(3))
scatter_matrix

low PM2.5 region ['Arizona' 'Colorado' 'Connecticut' 'Idaho' 'Massachusetts' 'Maine'
 'Minnesota' 'Montana' 'North Dakota' 'Nebraska' 'New Hampshire' 'Nevada'
 'New Mexico' 'Oregon' 'Rhode Island' 'South Dakota' 'Utah' 'Vermont'
 'Washington' 'Wisconsin' 'Wyoming']
=== Correlation Matrix (Pearson) ===


,COPD Prevalence (%),Smoking Rate (%),Diabetes Prevalence (%),Poverty Rate (%),"PM2,5 Level (μg/m³)"
COPD Prevalence (%),1.000,0.812,0.792,0.576,0.288
Smoking Rate (%),0.812,1.000,0.634,0.594,0.188
Diabetes Prevalence (%),0.792,0.634,1.000,0.666,0.494
Poverty Rate (%),0.576,0.594,0.666,1.000,0.405
"PM2,5 Level (μg/m³)",0.288,0.188,0.494,0.405,1.000


alt.RepeatChart(...)

# 4. Data Analysis

### Bayesian Hierarchical Regression for COPD rate Prediction

![model.jpg](model.jpg)

In [8]:
merged_df['State_idx'] = merged_df['LocationAbbr'].astype('category').cat.codes
state_idx = merged_df['State_idx'].values
n_states = merged_df['State_idx'].nunique()

X = merged_df[['Smoking_Rate', 'Diabetes_Rate', 'Poverty_Rate', 'PM25_Level']].values
X4_squared = X[:, 3] ** 2
Y = merged_df['COPD_Rate'].values
with pm.Model() as model:
    sigma_u = pm.HalfNormal('sigma_u', sigma=1)
    u = pm.Normal('u', mu=0, sigma=sigma_u, shape=n_states)
    beta0 = pm.Normal('beta0', mu=0, sigma=1)
    beta = pm.Normal('beta', mu=0, sigma=1, shape=4)
    beta_quad = pm.Normal('beta_quad', mu=0, sigma=1)
    sigma_Y = pm.HalfNormal('sigma_Y', sigma=1)
    mu = u[state_idx] + beta0 + pm.math.dot(X, beta) + beta_quad*X4_squared
    Y_obs = pm.Normal('Y_obs', mu=mu, sigma=sigma_Y, observed=Y)
    trace = pm.sample(random_seed=42)

fixed_summary = az.summary(trace, var_names=['beta0', 'beta', 'beta_quad', 'sigma_u', 'sigma_Y'])
state_mapping = dict(enumerate(merged_df['LocationAbbr'].astype('category').cat.categories))
u_summary = az.summary(trace, var_names=['u'])
u_summary.index = ['u['+state_mapping[int(s.split('[')[1].split(']')[0])]+']' for s in u_summary.index]
combined_summary = pd.concat([fixed_summary, u_summary])
display(combined_summary)

Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [sigma_u, u, beta0, beta, beta_quad, sigma_Y]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 43 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta0,0.265,0.846,-1.347,1.812,0.027,0.015,953.0,1538.0,1.00
beta[0],0.165,0.045,0.083,0.253,0.002,0.001,483.0,728.0,1.01
beta[1],0.460,0.054,0.359,0.559,0.002,0.001,759.0,1863.0,1.00
beta[2],0.066,0.047,-0.025,0.150,0.002,0.001,667.0,1389.0,1.01
beta[3],-0.449,0.251,-0.934,0.012,0.010,0.006,663.0,1094.0,1.00
beta_quad,0.026,0.017,-0.005,0.059,0.001,0.000,687.0,976.0,1.00
sigma_u,0.963,0.129,0.729,1.201,0.004,0.002,860.0,1791.0,1.01
sigma_Y,0.513,0.032,0.455,0.576,0.001,0.000,1188.0,1655.0,1.01
u[AL],0.724,0.355,0.046,1.395,0.015,0.007,596.0,1459.0,1.01
u[AR],0.689,0.351,0.007,1.309,0.013,0.006,692.0,1498.0,1.01


In [9]:
u_summary = az.summary(trace, var_names=['u'])
u_summary.index = [state_mapping[int(s.split('[')[1].split(']')[0])] for s in u_summary.index]
u_summary['State'] = u_summary.index
map_df = u_summary[['State', 'mean']].rename(columns={'mean': 'u_mean'})
states = alt.topo_feature(data.us_10m.url, 'states')
map_df['fips'] = map_df['State'].map(abbr_to_fips)
chart = alt.Chart(states).mark_geoshape().encode(
    color=alt.Color('u_mean:Q', scale=alt.Scale(scheme='reds'), title='Effect (%)'),
    tooltip=['State:N', 'u_mean:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(map_df, 'fips', ['State', 'u_mean'])
).project(
    type='albersUsa'
).properties(
    title='State-specific Effects (u)'
)
chart.display()

alt.Chart(...)

### Data Preparation: Extrapolate the 2020 data with Naive Linear Regression

In [10]:
def extrapolate_to_2020(df, year_col, value_col, group='LocationAbbr'):
    out = []
    for state, g in df.groupby(group):
        g = g.sort_values(year_col)
        years = g[year_col].values
        vals = g[value_col].values
        
        if len(years) > 2:
            a, b = np.polyfit(years, vals, 1)
            pred = a * 2020 + b
        else:
            pred = vals.mean()
        
        out.append({group: state, year_col: 2020, value_col: pred})
    return pd.DataFrame(out)
# PM2.5
pm25 = (
    pd.read_csv('Daily_Census_Tract-Level_PM2.5_Concentrations,_2016_-_2020_averaged_by_state.csv')
    .rename(columns={'DS_PM_pred': 'PM25_Level'})
)
pm25['LocationAbbr'] = pm25['statefips'].map(fips_to_abbr)
pm25_hist = pm25[pm25['year'].between(2016, 2019)]
pm25_2020 = extrapolate_to_2020(pm25_hist, 'year', 'PM25_Level')
# Smoking
smoking = (
    pd.read_csv('Behavioral_Risk_Factor_Surveillance_System_(BRFSS)_Prevalence_Data_(2011_to_present)_filtered_by_smoking.csv')
    .rename(columns={'Locationabbr': 'LocationAbbr', 'Data_value': 'Smoking_Rate'})
)
smoking = smoking[['LocationAbbr', 'Year', 'Smoking_Rate']]
smoking_2020 = extrapolate_to_2020(smoking[smoking['Year'] <= 2019], 'Year', 'Smoking_Rate')
# Poverty
poverty = pd.read_csv('poverty-rate-by-raceethnicity(2008_to_2023).csv', header=2)
poverty = poverty.melt(
    id_vars=['Location'],
    value_vars=[c for c in poverty.columns if '__Total' in c],
    var_name='Year_Type',
    value_name='Poverty_Rate'
)
poverty['Year'] = poverty['Year_Type'].str.split('__').str[0].astype(int)
poverty['Poverty_Rate'] *= 100
poverty['LocationAbbr'] = poverty['Location'].map(state_to_abbr)
poverty_2020 = extrapolate_to_2020(poverty, 'Year', 'Poverty_Rate')
# Diabetes
diabetes = (
    pd.read_csv('Behavioral_Risk_Factor_Surveillance_System_(BRFSS)_Prevalence_Data_(2011_to_present)_filtered_by_diabetes.csv')
    .rename(columns={'Locationabbr': 'LocationAbbr', 'Data_value': 'Diabetes_Rate'})
)
diabetes = diabetes[['LocationAbbr', 'Year', 'Diabetes_Rate']]
diabetes_2020 = extrapolate_to_2020(diabetes[diabetes['Year'] <= 2019], 'Year', 'Diabetes_Rate')
# Merge All Covariates for 2020
cov = (smoking_2020
       .merge(diabetes_2020, on='LocationAbbr')
       .merge(poverty_2020, on='LocationAbbr')
       .merge(pm25_2020, on='LocationAbbr'))

### 2020 COPD rate prediction with Bayesian Hierarchical Model

In [11]:
# 2020 COPD rate prediction with Bayesian Hierarchical Model
state_to_idx = {s: i for i, s in enumerate(cov['LocationAbbr'].unique())}
X = cov[['Smoking_Rate','Diabetes_Rate','Poverty_Rate','PM25_Level']].values
X_quad = X[:, 3]**2
state_idx = np.array([state_to_idx[s] for s in cov['LocationAbbr']])
posterior = az.summary(trace, var_names=['beta0','beta','beta_quad','u'])
beta0 = posterior.loc['beta0', 'mean']
beta = posterior.loc[['beta[0]', 'beta[1]', 'beta[2]', 'beta[3]'], 'mean'].values
beta_quad = posterior.loc['beta_quad', 'mean']
u_mean = trace.posterior['u'].mean(dim=['chain','draw']).values
mu_pred = u_mean[state_idx] + beta0 + X.dot(beta) + beta_quad * X_quad
copd_2020_pred = pd.DataFrame({'LocationAbbr': cov['LocationAbbr'], 'pred_COPD_2020': mu_pred})
# Compare with Actual COPD
copd = (
    pd.read_csv('U.S._Chronic_Disease_Indicators__CDI___2023_Release_filtered_by_COPD.csv')
    .rename(columns={'DataValue': 'actual_COPD_2020', 'YearStart': 'Year'})
)
copd = copd[copd['Year'] == 2020][['LocationAbbr','actual_COPD_2020']]
comparison = copd_2020_pred.merge(copd, on='LocationAbbr')
rmse = np.sqrt(((comparison['pred_COPD_2020'] - comparison['actual_COPD_2020'])**2).mean())
print(f"RMSE: {rmse:.4f}")
print(comparison.head())

RMSE: 0.5617
  LocationAbbr  pred_COPD_2020  actual_COPD_2020
0           AL       10.166827               9.9
1           AR       10.012172               9.0
2           AZ        6.647131               6.0
3           CA        4.543362               5.4
4           CO        4.214921               4.5


### 2020 COPD rate prediction with Naive Linear Regression

In [12]:
# copd prediction
copd = pd.read_csv('U.S._Chronic_Disease_Indicators__CDI___2023_Release_filtered_by_COPD.csv')
copd = copd.rename(columns={'DataValue':'COPD_Rate','YearStart':'Year'})
copd['LocationAbbr'] = copd['LocationDesc'].map(state_to_abbr)
train = copd[copd['Year'].between(2011, 2019)]
actual2020 = copd[copd['Year']==2020][['LocationAbbr','COPD_Rate']].rename(columns={'COPD_Rate':'actual_COPD_2020'})
def predict(g):
    g = g.sort_values('Year').dropna(subset=['COPD_Rate'])
    y, x = g.COPD_Rate.values, g.Year.values
    if len(g) > 2:
        m, b = np.polyfit(x, y, 1)
        return m*2020 + b
    return y.mean()
pred = train.groupby('LocationAbbr').apply(predict, include_groups=False).reset_index()
pred = pred.rename(columns={0:'pred_COPD_2020'})
# RMSE
cmp = pred.merge(actual2020, on='LocationAbbr')
rmse = np.sqrt(((cmp.actual_COPD_2020 - cmp.pred_COPD_2020)**2).mean())
print(f"RMSE: {rmse:.4f}")
print(cmp.head())

RMSE: 0.7767
  LocationAbbr  pred_COPD_2020  actual_COPD_2020
0           AL       10.336111               9.9
1           AR       10.625000               9.0
2           AZ        7.111111               6.0
3           CA        4.386111               5.4
4           CO        4.311111               4.5
